# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? False


In [1]:
import pandas as pd, yfinance as yf
from datetime import datetime, timezone
import pathlib

def nowstamp():
    return datetime.now(timezone.utc).strftime("%Y%m%d-%H%M")

def save_raw_csv(df, prefix, tail):
    pathlib.Path("../data/raw").mkdir(parents=True, exist_ok=True)
    out = f"../data/raw/{prefix}_{tail}_{nowstamp()}.csv"
    df.to_csv(out, index=False)
    print("Saved:", out)
    return out

SYMBOL = "AAPL"

df_api = yf.download(SYMBOL, period="1y", interval="1d").reset_index()
df_api = df_api.rename(columns={
    "Date":"date","Open":"open","High":"high","Low":"low",
    "Close":"close","Adj Close":"adj_close","Volume":"volume"
})

# validation
required = ["date","open","high","low","close","volume"]
assert all(c in df_api.columns for c in required), "Missing expected columns"
assert len(df_api) > 0, "No rows pulled"

# save
api_path = save_raw_csv(df_api, "api_yfinance", SYMBOL)
df_api.head()


/var/folders/bj/jr3h9pxd59v7pj8_wlgvgwf80000gn/T/ipykernel_89405/2027634466.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period="1y", interval="1d").reset_index()
[*********************100%***********************]  1 of 1 completed

Saved: ../data/raw/api_yfinance_AAPL_20250821-0046.csv


Price,date,close,high,low,open,volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL
0,2024-08-21,225.351212,226.923894,224.007474,225.470666,34765500
1,2024-08-22,223.489868,227.282216,222.862782,226.734761,43695300
2,2024-08-23,225.789185,227.162797,223.290817,224.614658,38677300
3,2024-08-26,226.127579,226.227122,222.852826,225.709526,30602200
4,2024-08-27,226.973663,227.789872,223.848210,224.953068,35934600


## Helpers (use or modify)

In [ ]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [ ]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

In [ ]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [1]:
# --- Scrape a public table (Wikipedia) and save to data/raw ---
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timezone
import pathlib

def nowstamp():
    return datetime.now(timezone.utc).strftime("%Y%m%d-%H%M")

# pick a permitted public table
SCRAPE_URL = "https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue"
headers = {"User-Agent": "AFE-Homework/1.0"}

try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "lxml")

    # parse rows (header row + data rows)
    rows = [[c.get_text(strip=True) for c in tr.find_all(["th","td"])]
            for tr in soup.find_all("tr")]
    rows = [r for r in rows if r]  # drop empties
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

except Exception as e:
    print("Scrape failed, using inline demo table:", e)
    html = "<table><tr><th>Name</th><th>Revenue(US$M)</th></tr><tr><td>DemoCo</td><td>101.2</td></tr></table>"
    soup = BeautifulSoup(html, "html.parser")
    rows = [[c.get_text(strip=True) for c in tr.find_all(["th","td"])]
            for tr in soup.find_all("tr")]
    rows = [r for r in rows if r]
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

# keep a tidy subset if present
keep = [c for c in ["Rank","Name","Industry","Revenue","Revenue(USD$bn)","Revenue(USD$ millions)"] if c in df_scrape.columns]
if keep:
    df_scrape = df_scrape[keep]

# basic validation
assert len(df_scrape) > 10, "Too few rows parsed"
print("Parsed shape:", df_scrape.shape)

# save to data/raw
pathlib.Path("../data/raw").mkdir(parents=True, exist_ok=True)
out_path = f"../data/raw/scrape_wikipedia_companies_{nowstamp()}.csv"
df_scrape.to_csv(out_path, index=False)
print("Saved:", out_path)

df_scrape.head()


Parsed shape: (79, 3)
Saved: ../data/raw/scrape_wikipedia_companies_20250821-0111.csv


,Name,Industry,Revenue
0,None,None,None
1,Walmart,Retail,"$680,985"
2,Amazon,Retailinformation technology,"$637,959"
3,State Grid Corporation of China,Electricity,"$545,948"
4,Saudi Aramco,Oil and gas,"$480,446"


In [ ]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.